## WattTime API
WattTime API 提供全球電網的即時數據、預測數據和歷史數據，包括邊際排放數據，其提供2種衡量電力碳排放的指標
- API 文檔: https://docs.watttime.org/
- 狀態頁面: https://status.watttime.org/
- Map: https://watttime.org/docs-dev/coverage-map/
- GitHub 範例: https://github.com/WattTime/watttime-python-client

### WattTime提供2種衡量電力碳排放的指標
1. 邊際運營排放率Marginal Operating Emissions Rate(MOER)：當電力負載增加或減少一單位時，電力系統排放的二氧化碳當量 CO2e會如何變化
  - 用途: 優化電力使用時機，將高耗能活動移至碳排放較低的時段；評估新增負載的真實影響
2. 平均運營排放率Average Operating Emissions Rate(AOER):在特定時間段內，整個電力系統每生產一單位電力（通常是每百萬瓦時 MWh 或每度電 kWh）所排放的平均溫室氣體量。
  - 用途: 計算企業或產品的碳足跡，相對靜態，通常以年或月為單位進行計算。

### 使用限制
查詢 API 的速率有嚴格限制。對於任何用戶，我們允許在每5分鐘內發送最多 3,000 個請求（平均每秒 10 個請求）

### Step1 註冊帳號
1. 透過 /register 端點註冊一個帳戶
2. 使用 /login 端點取得access token(30分鐘過期)
3. 取得可存取區域與API endpoint

In [ ]:
import requests

# 註冊新帳號
registration_url = "https://api2.watttime.org/v2/register"

# 填入使用者名稱與密碼，!!注意password (字母、數字和特殊字元各至少 1 個)
params = {
    "username": username,
    "password": password,
    "email": "richlee0423@gmail.com",
    "org": "rich"
}
response = requests.post(registration_url, json=params)
print(response.json())  # {'ok': 'User created'}

{'error': 'Failed to create user'}


In [1]:
from google.colab import userdata

username = userdata.get('USERNAME')
password = userdata.get('PASSWORD')

In [2]:
# 取得存取權杖
def get_token():
    login_url = "https://api2.watttime.org/v2/login"
    response = requests.get(login_url, auth=(username, password))
    return response.json()['token']

In [3]:
!pip install watttime

In [4]:
# 取得可存取區域與API endpoint
url = "https://api.watttime.org/v3/my-access"

TOKEN = get_token()
headers = {"Authorization": f"Bearer {TOKEN}"}
params = {}
response = requests.get(url, headers=headers, params=params)
response.raise_for_status()
print(response.json())

{'signal_types': [{'signal_type': 'co2_moer', 'regions': [{'region': 'CAISO_NORTH', 'region_full_name': 'California ISO Northern', 'parent': 'CAISO', 'data_point_period_seconds': 300, 'endpoints': [{'endpoint': 'v3/historical', 'models': [{'model': '2022-10-01', 'data_start': '2020-01-01', 'data_end': '2024-12-04', 'train_start': '2020-03-01', 'train_end': '2022-03-01', 'type': 'binned_regression'}, {'model': '2023-03-01', 'data_start': '2020-03-01', 'data_end': None, 'train_start': '2021-01-01', 'train_end': '2023-03-01', 'type': 'binned_regression'}]}, {'endpoint': 'v3/forecast', 'models': [{'model': '2022-10-01', 'data_start': '2024-08-20', 'data_end': '2024-11-20', 'train_start': '2020-01-01', 'train_end': '2021-12-31', 'historical_model': '2022-10-01'}, {'model': '2023-03-01', 'data_start': '2024-08-20', 'data_end': None, 'train_start': '2021-01-01', 'train_end': '2023-01-01', 'historical_model': '2023-03-01'}]}]}]}, {'signal_type': 'health_damage', 'regions': [{'region': 'CAISO_N

### Step2 取得碳強度預測


In [5]:
# 取得未來 24 小時預測
def get_forecast(ba_code):
    url = "https://api.watttime.org/v3/forecast"
    params = {
      "region": ba_code,
      "signal_type": "co2_moer",
    }
    headers = {"Authorization": f"Bearer {get_token()}"}
    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        forecast_data = response.json()["data"]
        print(forecast_data)

        # 解析預測數據
        if forecast_data and len(forecast_data) > 0:
            forecasts = []
            for item in forecast_data:
                forecasts.append({
                    "timestamp": item["point_time"],
                    "value": item["value"]
                })
            return forecasts
        else:
            print("No forecast data available")
            return []

    except requests.exceptions.RequestException as e:
        print(f"API Error: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Response content: {e.response.text}")
        return []

In [ ]:
get_forecast("CAISO_NORTH")

In [8]:
# 找出最佳執行時段
def find_best_time_window(ba_code, duration_hours=2):
    forecasts = get_forecast(ba_code)

    if not forecasts:
        return None

    # 計算移動平均找出最低碳強度時段
    best_window = {"start": None, "avg_intensity": float('inf')}

    # 預測數據通常是 5 分鐘間隔
    window_size = duration_hours * 12  # 每小時 12 個 5 分鐘間隔

    for i in range(len(forecasts) - window_size + 1):
        window_data = forecasts[i:i + window_size]
        avg_intensity = sum(f["value"] for f in window_data) / len(window_data)

        if avg_intensity < best_window["avg_intensity"]:
            best_window = {
                "start": window_data[0]["timestamp"],
                "end": window_data[-1]["timestamp"],
                "avg_intensity": avg_intensity
            }

    return best_window
find_best_time_window("CAISO_NORTH")

[{'point_time': '2025-07-02T06:10:00+00:00', 'value': 945.5}, {'point_time': '2025-07-02T06:15:00+00:00', 'value': 945.5}, {'point_time': '2025-07-02T06:20:00+00:00', 'value': 943.0}, {'point_time': '2025-07-02T06:25:00+00:00', 'value': 944.1}, {'point_time': '2025-07-02T06:30:00+00:00', 'value': 944.6}, {'point_time': '2025-07-02T06:35:00+00:00', 'value': 943.7}, {'point_time': '2025-07-02T06:40:00+00:00', 'value': 940.6}, {'point_time': '2025-07-02T06:45:00+00:00', 'value': 939.6}, {'point_time': '2025-07-02T06:50:00+00:00', 'value': 941.6}, {'point_time': '2025-07-02T06:55:00+00:00', 'value': 937.5}, {'point_time': '2025-07-02T07:00:00+00:00', 'value': 935.0}, {'point_time': '2025-07-02T07:05:00+00:00', 'value': 932.7}, {'point_time': '2025-07-02T07:10:00+00:00', 'value': 932.7}, {'point_time': '2025-07-02T07:15:00+00:00', 'value': 933.6}, {'point_time': '2025-07-02T07:20:00+00:00', 'value': 932.9}, {'point_time': '2025-07-02T07:25:00+00:00', 'value': 934.3}, {'point_time': '2025-07

{'start': '2025-07-02T18:00:00+00:00',
 'end': '2025-07-02T19:55:00+00:00',
 'avg_intensity': 218.9375}

### 邊際碳排 (Marginal Operating Emissions Rate, MOER) Index
衡量電網在特定時間點，每當電力需求增加或減少一單位時，所導致的二氧化碳排放量變化
- percent: 0-100 的相對潔淨度（100 = 最乾淨）

In [10]:
url = "https://api.watttime.org/v3/signal-index"

# Provide your TOKEN here, see https://docs.watttime.org/#tag/Authentication/operation/get_token_login_get for more information
TOKEN = get_token()
headers = {"Authorization": f"Bearer {TOKEN}"}
params = {
    "region": "CAISO_NORTH",
    "signal_type": "co2_moer",
}
response = requests.get(url, headers=headers, params=params)
response.raise_for_status()
print(response.json())

{'data': [{'point_time': '2025-07-02T06:30:00+00:00', 'value': 87.0}], 'meta': {'data_point_period_seconds': 300, 'region': 'CAISO_NORTH', 'signal_type': 'co2_moer', 'units': 'percentile', 'warnings': [], 'model': {'date': '2023-03-01'}}}
